In [1]:
!pip install av

In [2]:
import torch
import torchvision

from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class Model(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.model = torchvision.models.video.swin3d_t()
    self.fc1 = nn.Linear(400, 100)
    self.fc2 = nn.Linear(100, num_classes)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x):
    x = self.model(x)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.softmax(x)
    return x
# model = torchvision.models.video.swin3d_t().to(device)

model = Model(num_classes=2).to(device)

In [4]:
video, _, _ = torchvision.io.read_video('video.avi')
print(f'Входной размер видоса: {video.shape}')
FRAMES = video.shape[0]
gray = torchvision.transforms.Grayscale()  # gray и resize тут просто, чтобы видос остался с одним каналом цвета
resize = torchvision.transforms.Resize((256, 256))
video = video.permute((3, 0, 1, 2))
video = resize(video).float().unsqueeze(dim=0).to(device)
print(video.shape)

/usr/local/lib/python3.10/dist-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Входной размер видоса: torch.Size([96, 400, 200, 3])
torch.Size([1, 3, 96, 256, 256])


In [9]:
model(video).shape

torch.Size([1, 2])